# Inference

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import datetime
import time
from pathlib import Path

import numpy as np
from PIL import Image
from sam2util import convert_jpg_to_mp4
from tqdm import tqdm

from model.inference import (
    init_dagshub,
    load_image,
    load_model_from_dagshub,
    measure_model_fps,
    predict,
)
from model.plot import plot_single_prediction


In [3]:
init_dagshub(repo_name='driver-state')

Accessing as matejfric

INFO:dagshub:Accessing as matejfric


Initialized MLflow to track repo "matejfric/driver-state"

INFO:dagshub:Initialized MLflow to track repo "matejfric/driver-state"


Repository matejfric/driver-state initialized!

INFO:dagshub:Repository matejfric/driver-state initialized!


In [6]:
MODEL_NAME = 'pytorch-2024-09-15-215831-unet-efficientnet-b0'
VERSION = 1
model = load_model_from_dagshub(MODEL_NAME, VERSION)

In [18]:
OUTPUT_DIR = Path('outputs') / f'{datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")}-{MODEL_NAME}-v{VERSION}'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

DATASET_PATH = Path('/home/lanter/source/driver-dataset/images/2022_09_14_stribny_enyaq/normal')

for image_path in tqdm(sorted(DATASET_PATH.glob('*.jpg'))):
    image = load_image(image_path)
    mask = predict(model, image, input_size=256, output_size=(1024, 1024))
    # plot_single_prediction(image, mask)

    mask = mask.squeeze().numpy()
    mask_rgba = np.zeros((*mask.shape, 4), dtype=np.uint8)
    mask_rgba[..., 0] = 255  # Red
    mask_rgba[..., 3] = (mask > 0.5) * 102  # Alpha channel with transparency
    mask_img = Image.fromarray(mask_rgba, "RGBA")
    image_rgba = image.resize((mask.shape[0], mask.shape[1])).convert("RGBA")
    overlay_img = Image.alpha_composite(image_rgba, mask_img)
    overlay_img.convert("RGB").save(OUTPUT_DIR / f'{int(image_path.stem) // 30 :05d}.jpg')

    with open(OUTPUT_DIR / 'dataset_path.txt', 'w') as f:
        f.write(str(DATASET_PATH))

100%|██████████| 476/476 [00:23<00:00, 20.30it/s]


In [15]:
measure_model_fps(model, DATASET_PATH)

Processed 476 images.
Total image loading time: 0.06 seconds
Total inference time: 8.90 seconds
Image loading FPS: 7989.60
Inference FPS: 53.51


In [23]:
DATASET_PATH.parent.name, DATASET_PATH.name

('2022_09_14_stribny_enyaq', 'normal')

In [24]:
# TODO: Implement INTERPOLATION (see Sapiens) to arbitrary output size!
FPS = 15
output_video_name = f'{DATASET_PATH.parent.name}_{DATASET_PATH.name}_{FPS}fps.mp4'
convert_jpg_to_mp4(OUTPUT_DIR, OUTPUT_DIR / output_video_name, fps=FPS)